<a href="https://colab.research.google.com/github/alievk/avatarify/blob/feat%2Fcolab-mode/avatarify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [0]:
!cd /content
!rm -rf *

In [0]:
!git clone https://github.com/alievk/avatarify.git -b feat/colab-mode

In [0]:
cd avatarify

In [0]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy

In [0]:
!scripts/download_data.sh

# Run

In [0]:
cd /content/avatarify

In [0]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [0]:
local_in_port = 5557
local_out_port = 5558

# Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [0]:
# Paste your authtoken here in quotes
authtoken = "1btOHLi8Xwo3QXFClnUCD99MN0s_3fvBGN7z7iinWdiyU5Xuw"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [0]:
# Set your region here in quotes
region = "eu"

In [0]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [0]:
!cat ngrok.conf

# Start the worker

In [0]:
# (Re)Start the worker
log = open('log.txt', 'w')
ps = Popen(shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port}'),
           stdout=log, stderr=log)
time.sleep(3)

This command should print lines if the worker is successfully started

In [0]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep

In [0]:
#!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#!apt install -qq -o=Dpkg::Use-Pty=0 netcat > /dev/null

In [0]:
cd /content/avatarify

In [0]:
!chmod +x open_aws_tunnel.sh

In [0]:
!./open_aws_tunnel.sh&

In [0]:
!ps aux | grep "ssh -f"

# Open tunnel

In [0]:
# (Re)Open tunnel
ps = Popen('scripts/open_tunnel.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [0]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
except:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

# Start the client
Run the cell below, it will print a command. Paste this command in your Mac terminal and run.

In [0]:
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

In [0]:
!cat log.txt | head -100

In [0]:
!cat recv_worker.log | tail -100

In [0]:
!cat predictor_worker.log | tail -100

In [0]:
!cat send_worker.log | tail -100